# Install

```commandline
conda create -c conda-forge -n dpz gdal python=3.6 jupyter gdal jupyter_contrib_nbextensions -y
conda activate dpz
```

# First steps

In [8]:
import gdal
dataset = gdal.Open("c:/Users/Jirik/data/dpz_data/dpz_data_erdas/2-2.img")
dataset.GetMetadata()
dataset.GetRasterBand(1)

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x000002C6BCC148D0> >

## Getting Dataset Information

In [9]:
print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                            dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))
geotransform = dataset.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: HFA/Erdas Imagine Images (.img)
Size is 1403 x 1376 x 3
Projection is 
Origin = (1062350.0, 2062360.0)
Pixel Size = (8.0, -8.0)


## Fetching raster band

In [11]:
band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))

if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))

if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

Band Type=Byte
Min=5.000, Max=253.000
Band has 4 overviews


## Reading raster data

In [15]:
scanline = band.ReadRaster(xoff=0, yoff=0,
                        xsize=band.XSize, ysize=1,
                        buf_xsize=band.XSize, buf_ysize=1,
                        buf_type=gdal.GDT_Float32)

In [16]:
scanline

b'\x00\x00\xb2B\x00\x00\xbaB\x00\x00\xc4B\x00\x00\xccB\x00\x00\xbcB\x00\x00\xb4B\x00\x00\xc6B\x00\x00\xbaB\x00\x00\xc4B\x00\x00\xb6B\x00\x00\xbcB\x00\x00\xa4B\x00\x00\xa0B\x00\x00\x9aB\x00\x00\xa0B\x00\x00\xa0B\x00\x00\xa8B\x00\x00\xb0B\x00\x00\xc0B\x00\x00\xd2B\x00\x00\xdaB\x00\x00\xd2B\x00\x00\xceB\x00\x00\xd0B\x00\x00\xdaB\x00\x00\xe2B\x00\x00\xdcB\x00\x00\xcaB\x00\x00\xc4B\x00\x00\xa8B\x00\x00\xb8B\x00\x00\xacB\x00\x00\xaeB\x00\x00\xacB\x00\x00\x9cB\x00\x00\x8cB\x00\x00\xa0B\x00\x00\xb2B\x00\x00\xb4B\x00\x00\xbcB\x00\x00\xb6B\x00\x00\xa6B\x00\x00\xaaB\x00\x00\xa8B\x00\x00\xa0B\x00\x00\xa4B\x00\x00\x9aB\x00\x00\xa4B\x00\x00\xb0B\x00\x00\xaeB\x00\x00\xacB\x00\x00\xb6B\x00\x00\xc0B\x00\x00\xccB\x00\x00\xecB\x00\x00\xe8B\x00\x00\xfaB\x00\x00\xf2B\x00\x00\xf6B\x00\x00\xf2B\x00\x00\xfeB\x00\x00\xfaB\x00\x00\x01C\x00\x00\xeaB\x00\x00\xd0B\x00\x00\xc2B\x00\x00\xeeB\x00\x00\xf2B\x00\x00\xf6B\x00\x00\xe6B\x00\x00\xf4B\x00\x00\xf0B\x00\x00\xe0B\x00\x00\xf8B\x00\x00\xfeB\x00\x00\xfaB\x00\x00\x

In [19]:
# import struct
# tuple_of_floats = struct.unpack('f' * b2.XSize, scanline)

In [18]:
fileformat = "GTiff"
driver = gdal.GetDriverByName(fileformat)
metadata = driver.GetMetadata()
if metadata.get(gdal.DCAP_CREATE) == "YES":
    print("Driver {} supports Create() method.".format(fileformat))

if metadata.get(gdal.DCAP_CREATE) == "YES":
    print("Driver {} supports CreateCopy() method.".format(fileformat))

Driver GTiff supports Create() method.
Driver GTiff supports CreateCopy() method.


In [ ]:
def imageToArray(i):
    """
    Converts a Python Imaging Library array to a
    gdalnumeric image.
    """
    a=gdalnumeric.fromstring(i.tostring(),'b')
    a.shape=i.im.size[1], i.im.size[0]
    return a

In [21]:
import rasterio

# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(filepath) as src:
   oviews = src.overviews(1) # list of overviews from biggest to smallest
   oview = oviews[-1] # let's look at the smallest thumbnail
   print('Decimation factor= {}'.format(oview))
   # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
   thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

ModuleNotFoundError: No module named 'rasterio'